In [33]:
# ======================================================
# CONDITIONAL RANDOM FIELD (CRF) MODEL FOR CPT DATA
# ======================================================
# Author: Dorothy Chepkoech
# Purpose: Predict lithostratigraphic units from CPT data using CRF
# ======================================================

In [21]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm
import pycrfsuite
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
import paths_cpt 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc, RocCurveDisplay, silhouette_score
import os
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from seglearn.transform import FeatureRep, SegmentX
from seglearn.pipe import Pype
from seglearn.datasets import load_watch
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import random
df = pd.read_parquet(paths_cpt.PATH_TO_PARQUET)
print(df.shape)
df.dropna(inplace=True)
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(1222711, 19)


,sondering_id,index,pkey_sondering,sondeernummer,x,y,start_sondering_mtaw,diepte_sondering_tot,diepte,diepte_mtaw,qc,fs,qtn,rf,fr,icn,sbt,ksbt,lithostrat_id
0,314,2593,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.6,13.66,1.17,0.035,35.894004,2.991453,3.058371,2.564340,5.0,1.434000e-07,Quartair
1,314,2594,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.7,13.56,1.57,0.033,42.562319,2.101911,2.138968,2.406724,5.0,4.321000e-07,Quartair
2,314,2595,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.8,13.46,1.43,0.036,38.536991,2.517483,2.569226,2.491219,5.0,2.392000e-07,Quartair
3,314,2596,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.9,13.36,0.50,0.024,15.678501,4.800000,5.111166,2.982185,3.0,7.700000e-09,Quartair
4,314,2597,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,2.0,13.26,1.33,0.023,33.203119,1.729323,1.772110,2.440158,5.0,3.419000e-07,Quartair


In [22]:
# --- Step 2: Encode labels ---
# Encode the lithostratigraphic unit (target)
le = LabelEncoder()
df['litho_encoded'] = le.fit_transform(df['lithostrat_id'])

# --- Step 3: Define sequence segmentation ---
# Group by CPT ID ('sondeernummer') — each CPT is a sequence
grouped = df.groupby('sondeernummer')


In [24]:

def create_sequences(group):
    """Split grouped CPT data into sequences of features and labels."""
    X_seq = []
    y_seq = []
    for _, row in group.iterrows():
        X_seq.append({
            'qc': row['qc'],
            'fs': row['fs'],
            'rf': row['rf'],
            'depth': row['diepte']
        })
        y_seq.append(row['litho_encoded'])
    return X_seq, y_seq

sequences = [create_sequences(g) for _, g in grouped]
X_sequences, y_sequences = zip(*sequences)

print(f"Total CPTs (sequences): {len(X_sequences)}")
print(f"Example sequence length: {len(X_sequences[0])}")


Total CPTs (sequences): 215
Example sequence length: 868


In [26]:
# --- Step 4: Feature extraction with context ---
def extract_features(seq, i):
    """
    Extracts geological + contextual features for each CPT measurement.
    Adds previous/next depth info for CRF sequence modeling.
    """
    features = {
        'qc': seq[i]['qc'],
        'fs': seq[i]['fs'],
        'rf': seq[i]['rf'],
        'depth': seq[i]['depth'],
        'depth_bin': int(seq[i]['depth'] // 2),  # binning depth by 2 meters
    }

    # Previous layer context
    if i > 0:
        features.update({
            'prev_qc': seq[i-1]['qc'],
            'prev_rf': seq[i-1]['rf'],
            'prev_depth_bin': int(seq[i-1]['depth'] // 2)
        })
    else:
        features['BOS'] = True  # Beginning of sequence

    # Next layer context
    if i < len(seq) - 1:
        features.update({
            'next_qc': seq[i+1]['qc'],
            'next_rf': seq[i+1]['rf'],
            'next_depth_bin': int(seq[i+1]['depth'] // 2)
        })
    else:
        features['EOS'] = True  # End of sequence

    return features

In [27]:
def transform_for_crf(X_seqs, y_seqs):
    """Convert data into CRF-ready format (lists of feature dicts + labels)."""
    X = [[extract_features(seq, i) for i in range(len(seq))] for seq in X_seqs]
    y = [[str(label) for label in seq] for seq in y_seqs]  # ensure string labels for CRF
    return X, y

X, y = transform_for_crf(X_sequences, y_sequences)

# --- Step 5: Split train/test sequences ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)
print(f"Training sequences: {len(X_train)}, Test sequences: {len(X_test)}")

Training sequences: 150, Test sequences: 65


In [28]:

# --- Step 6: Train CRF model ---
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in tqdm(zip(X_train, y_train), total=len(X_train), desc="Training CRF"):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 0.1,  # L1 regularization
    'c2': 0.1,  # L2 regularization
    'max_iterations': 200,
    'feature.possible_transitions': True
})

trainer.train('litho_crf_model.crfsuite')
print("Model training complete and saved as 'litho_crf_model.crfsuite'")

Training CRF: 100%|██████████| 150/150 [00:00<00:00, 655.57it/s]


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 928
Seconds required: 0.041

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 307450.434049
Feature norm: 0.250000
Error norm: 896973.095320
Active features: 927
Line search trials: 3
Line search step: 0.000000
Seconds required for this iteration: 0.895

***** Iteration #2 *****
Loss: 274855.497118
Feature norm: 0.244330
Error norm: 401618.856346
Active features: 908
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.174

***** Iteration #3 *****
Loss: 268354.590181
Feature norm: 0.234189
Error norm: 201916.643840
Active features: 928
Line search trials: 1
Line search step: 1.000000
Seconds required for t

In [29]:
# --- Step 7: Evaluate model ---
tagger = pycrfsuite.Tagger()
tagger.open('litho_crf_model.crfsuite')

y_pred = [tagger.tag(xseq) for xseq in X_test]

# Flatten sequences for evaluation
y_true_flat = np.concatenate(y_test)
y_pred_flat = np.concatenate(y_pred)

In [31]:
# --- Step 8: Classification report ---
from sklearn.metrics import classification_report

print("Classification Report:")

# Get unique labels from the actual data
unique_labels = np.unique(np.concatenate([y_true_flat, y_pred_flat]))

# Generate the report using only those labels
print(classification_report(
    y_true_flat,
    y_pred_flat,
    labels=unique_labels,
    target_names=[le.classes_[int(i)] for i in unique_labels if int(i) < len(le.classes_)],
    zero_division=0
))


Classification Report:
                                precision    recall  f1-score   support

                       Aalbeke       0.48      0.45      0.47      1301
                   Antropogeen       0.00      0.00      0.00       485
                          Lede       0.68      0.29      0.40      8828
                      Maldegem       0.00      0.00      0.00        59
                     Merelbeke       0.00      0.00      0.00       152
                Mons_en_Pevele       0.09      0.22      0.13      2289
                  Mont_Panisel       0.44      0.59      0.51      3337
                      Onbekend       0.60      0.22      0.32      6601
                       Orchies       0.00      0.00      0.00       250
                      Quartair       0.56      0.62      0.59     13667
       Quartair + Mont_Panisel       0.00      0.00      0.00       665
             Quartair_Holoceen       0.00      0.00      0.00         0
                          Asse       0.3